## Demostrator 1
#### Purpose:
- Load and explore a target variable from ERA5 used by the IceNet2 model

#### Comments:
- Some code snippets were adapted from icenet2 repository authored by Tom Ardersoon

#### Author: 
Alejandro Coca-Castro

## Set the environment

In [1]:
# Import the required packages
import virtualenv
import pip
import os

# Define and create the base directory install virtual environments
venvs_dir = os.path.join(os.path.expanduser("~"), "nb-venvs")

if not os.path.isdir(venvs_dir):
    os.makedirs(venvs_dir)
    
# Define the venv directory
venv_dir = os.path.join(venvs_dir, 'venv-cdi')

if not os.path.exists(venv_dir):
    # Create the virtual environment
    virtualenv.create_environment(venv_dir)
    
    # Install a set of required packages via `pip`
    requirements = ['cdsapi', 'intake']

    for pkg in requirements:
        pip.main(["install", "--prefix", venv_dir, pkg])
        
# Activate the venv
activate_file = os.path.join(venv_dir, "bin", "activate_this.py")
exec(open(activate_file).read(), dict(__file__=activate_file))

## Load libraries

In [2]:
import cdsapi
import xarray as xr
import numpy as np
import time
import matplotlib.pyplot as plt

## Request ERA5 data from CDI API

In [10]:
cds = cdsapi.Client()

In [4]:
# Settings
## Variables
variables = {
    'tas': {
        'cdi_name': '2m_temperature',
    },
    'tos': {
        'cdi_name': 'sea_surface_temperature',
    },
    'ta500': {
        'plevel': '500',
        'cdi_name': 'temperature',
    },
}

## Target months, days and times
months = [
    '01', '02', '03', '04', '05', '06',
    '07', '08', '09', '10', '11', '12'
]

days = [
    '01', '02', '03',
    '04', '05', '06',
    '07', '08', '09',
    '10', '11', '12',
    '13', '14', '15',
    '16', '17', '18',
    '19', '20', '21',
    '22', '23', '24',
    '25', '26', '27',
    '28', '29', '30',
    '31',
]

times = [
    '00:00', '01:00', '02:00',
    '03:00', '04:00', '05:00',
    '06:00', '07:00', '08:00',
    '09:00', '10:00', '11:00',
    '12:00', '13:00', '14:00',
    '15:00', '16:00', '17:00',
    '18:00', '19:00', '20:00',
    '21:00', '22:00', '23:00',
]

In [5]:
# Variables
var = 'ta500'
hemisphere = 'nh'
year = 2020

# Output folder and files
var_folder = os.path.join(os.path.expanduser("~"),'data', hemisphere, var)
if not os.path.exists(var_folder):
    os.makedirs(var_folder)
    
download_path = os.path.join(var_folder, '{}_latlon_hourly_{}_{}.nc'.format(var, year, year))
daily_fpath = os.path.join(var_folder, '{}_latlon_{}_{}.nc'.format(var, year, year))

In [7]:
# Settings 
if hemisphere == 'nh':
    area = [90, -180, 0, 180]
elif hemisphere == 'sh':
    area = [-90, -180, 0, 180]
elif hemisphere == 'nh_sh':
    area = [-90, -180, 90, 180]

var_dict = variables[var]

retrieve_dict = {
    'product_type': 'reanalysis',
    'variable': var_dict['cdi_name'],
    'year': year,
    'month': months,
    'day': days,
    'time': times,
    'format': 'netcdf',
    'area': area
}

if 'plevel' not in var_dict.keys():
    dataset_str = 'reanalysis-era5-single-levels'

elif 'plevel' in var_dict.keys():
    dataset_str = 'reanalysis-era5-pressure-levels'
    retrieve_dict['pressure_level'] = var_dict['plevel']

In [ ]:
present_time = time.time()
cds.retrieve(dataset_str, retrieve_dict, download_path)
print("--- %s seconds ---" % (time.time() - start_time))
print('\n\nDownload completed.')

2021-05-14 20:58:48,263 INFO Welcome to the CDS
2021-05-14 20:58:48,266 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2021-05-14 20:58:48,301 INFO Request is queued
2021-05-14 20:58:53,109 INFO Request is running


## Load and resample from hourly to daily

In [ ]:
da = xr.open_dataarray(download_path)

In [ ]:
print('\n\nComputing daily averages... ', end='', flush=True)
da_daily = da.resample(time='1D').reduce(np.mean)

if var == 'zg500' or var == 'zg250':
    da_daily = da_daily / 9.80665

if var == 'tos':
    # Replace every value outside of SST < 1000 with zeros (the ERA5 masked values)
    da_daily = da_daily.where(da_daily < 1000., 0)

In [ ]:
print(np.unique(da_daily.data)[[0, -1]])  # TEMP
print('saving new daily year file... ', end='', flush=True)
da_daily.to_netcdf(daily_fpath)

## Visualization

In [ ]:
# plot the last timestep
da.isel(time=-1).plot()

In [ ]:
# plot a timeseries at one location
da.sel(longitude=114.3055, latitude=30.5928, method='nearest').plot()